<a href="https://colab.research.google.com/github/ted-the-agi/PythonCodes/blob/main/AdvancedPythonAuthentication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title LOGIN

import csv
import hashlib
import requests
import base64
import sqlite3

DATABASE_URL_ENCODED = "base64_link_to_the_csv_file"
EXPECTED_CHECKSUM = "md5_checksum_of_the_file"

def download_database(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.content
    else:
        raise Exception("Failed to download the database file.")

def decode_database_url():
    encoded_url = base64.b64decode(DATABASE_URL_ENCODED).decode("utf-8")
    return encoded_url

def create_database():
    conn = sqlite3.connect(":memory:")  # Create an in-memory database
    return conn

def populate_database(conn, data):
    cursor = conn.cursor()
    query = "CREATE TABLE users (username TEXT, password_hash TEXT);"
    cursor.execute(query)

    for row in data:
        if len(row) >= 2:
            username = row[0]
            password_hash = row[1]
            query = "INSERT INTO users (username, password_hash) VALUES (?, ?);"
            cursor.execute(query, (username, password_hash))

    conn.commit()

def login():
    username = "" #@param {type:"string"}
    password = "" #@param {type:"string"}
    password_hash = hashlib.sha256(password.encode()).hexdigest()

    database_url = decode_database_url()
    database_content = download_database(database_url)
    database_content = database_content.decode("utf-8").splitlines()
    reader = csv.reader(database_content)

    conn = create_database()
    populate_database(conn, reader)

    cursor = conn.cursor()
    query = "SELECT * FROM users WHERE username = ? AND password_hash = ?"
    cursor.execute(query, (username, password_hash))
    row = cursor.fetchone()
    if row:
        print("Login successful!")
    else:
        print("Invalid username or password.")

    conn.close()

login()


Login successful!
